In [63]:
import math
import numpy as np
import pandas as pd

# Project 3: **The Hartree-Fock self-consistent field (SCF) procedure**


This project is a simple implementation of the self-consistent-field method. 

## **Integral Preprocessing**

In [64]:
molecule = "H\u2082O"

N = 10 #number of total electrons

n_basis = np.array([5,1,1]) # we have 5 AOs on 1st atom (O), 1 AOs on 2nd atom (H), 1 AOs on 3rd atom (H)

### Step 1: **Nuclear Repulsion Energy**

Read the nuclear repulsion energy $E_{nuc}$ 

In [65]:
E_nuc = np.genfromtxt("enuc.dat.txt")

### Step 2: **One-Electron Integrals**

Read the AO-basis overlap $$S_{\mu\nu}=\int \phi_\mu^*(\mathbf{r})\phi_\nu(\mathbf{r})d\mathbf{r}$$ kinetic-energy $$T_{\mu\nu}=\int \phi_\mu^*(\mathbf{r}) \left(-\frac{1}{2}\nabla_r^2 \right) \phi_\nu(\mathbf{r})d\mathbf{r}$$ nuclear-attraction integrals $$V_{\mu\nu}=\int \phi_\mu^*(\mathbf{r}) \left(-\sum_A^N\frac{Z}{r_A} \right) \phi_\nu(\mathbf{r})d\mathbf{r}$$ and store them in appropriately constructed matrices. 
$$$$
Then form the "core Hamiltonian" $$H_{\mu\nu}^{core}=T_{\mu\nu} +V_{\mu\nu}$$
$$$$
Note that the one-electron integrals provided include only the permutationally unique integrals, in the format $$\begin{matrix} 1 & 1 & A_{11} \\ 2 & 1 & A_{21} \\ 2 & 2 & A_{22} \\ 3 & 1 & A_{31} \\ \vdots & \ \ \vdots & \ \ \vdots \ \ \\ 7 & 7 & A_{77} \end{matrix}$$ but we should store the full matrices for convenience. $$\begin{matrix} A_{11} & \color{gray}{A_{21}} & \color{gray}{A_{31}} & \color{gray}{\cdots} & \color{gray}{A_{71}} \\ A_{21} & A_{22} & \color{gray}{A_{32}} & \color{gray}{\cdots} & \color{gray}{A_{72}} \\ A_{31} & A_{32} & A_{33} & \color{gray}{\cdots} & \color{gray}{A_{73}} \\ \vdots & \vdots & \vdots & \ddots & \color{gray}{\vdots} \\ A_{71} & A_{72} & A_{73} & \cdots & A_{77}\end{matrix} % usually I'd have used \textcolor instead of \color $$

Note also that the AO indices on the integrals in the files start with "$\ \ _1$" rather than "$\ \ _0$".
 

In [66]:
Smunu = np.genfromtxt("s.dat.txt")
Tmunu = np.genfromtxt("t.dat.txt")
Vmunu = np.genfromtxt("v.dat.txt")

nAO = int(np.max(Smunu[:,0])) # to get the number of AO, pick-up the max value from 0th column of Smunu
# could've got it from 0th/1th column of Smunu/Tmunu/Vmunu, or even 0th/1th/2th/3th column of eri 


def make_2Dmatrix(X):
  A = np.zeros([nAO,nAO])
  for a in range(len(X)): # it goes over all the lines of the input array
    mu=int(X[a,0]-1)
    nu=int(X[a,1]-1)
    A[mu,nu]=X[a,2]
    A[nu,mu]=X[a,2]
  return A

S = make_2Dmatrix(Smunu)
T = make_2Dmatrix(Tmunu)
V = make_2Dmatrix(Vmunu)

def add_matrices(X,Y):
  Z = np.zeros([len(X),len(X)])
  for i in range(len(X)):
    for j in range(len(X)):
      Z[i,j] = X[i,j]+Y[i,j]
  return Z

H_core = add_matrices(T,V)

def my_print(x):
    for row in x:
        print(' '.join(map(lambda x: "{:11.7f}".format(x), row)))

print("\n The core Hamiltonian for the ", molecule, " test case is \n")
my_print(H_core)
print("\n",end="")


 The core Hamiltonian for the  H₂O  test case is 

-32.5773954  -7.5788328   0.0000000  -0.0144738   0.0000000  -1.2401023  -1.2401023
 -7.5788328  -9.2009433   0.0000000  -0.1768902   0.0000000  -2.9067098  -2.9067098
  0.0000000   0.0000000  -7.4588193   0.0000000   0.0000000  -1.6751501   1.6751501
 -0.0144738  -0.1768902   0.0000000  -7.4153118   0.0000000  -1.3568683  -1.3568683
  0.0000000   0.0000000   0.0000000   0.0000000  -7.3471449   0.0000000   0.0000000
 -1.2401023  -2.9067098  -1.6751501  -1.3568683   0.0000000  -4.5401711  -1.0711459
 -1.2401023  -2.9067098   1.6751501  -1.3568683   0.0000000  -1.0711459  -4.5401711



### Step 3: **Two-Electron Integrals**

Read the two-electron repulsion integrals $$(\mu\nu|\lambda\sigma) = \int \phi_\mu^*(\mathbf{r}_1)\phi_\nu(\mathbf{r}_1)\phi_\lambda^*(\mathbf{r}_2)\phi_\sigma(\mathbf{r}_2)d\mathbf{r}_1 d\mathbf{r}_2$$ Hence, the integrals obey the eight-fold permutational symmetry relationships $$(\mu\nu|\lambda\sigma) = (\nu\mu|\lambda\sigma) = (\mu\nu|\sigma\lambda) = (\nu\mu|\sigma\lambda) = (\lambda\sigma|\mu\nu) = (\sigma\lambda|\mu\nu) = (\lambda\sigma|\nu\mu) = (\sigma\lambda|\nu\mu)$$
$$$$
Note that only the permutationally unique integrals are provided in the file, in the format (not necessarily in the same order) $$\begin{matrix} 1 & 1 & 1 & 1 & A_{1111}\\ 2 & 1 & 1 & 1 & A_{2111} \\ 2 & 2 & 1 & 1 & A_{2211} \\ 2 & 1 & 2 & 1 & A_{2121} \\ 2 & 2 & 2 & 2 & A_{2222} \\ \vdots & \vdots & \vdots & \vdots & \vdots \end{matrix}$$ but we should store the full 4-dimensional tensor.

Note also that the AO indices on the integrals in the files start with "$\ \ _1$" rather than "$\ \ _0$".

In [67]:
eri = np.genfromtxt("eri.dat.txt")

def make_4Dtensor(X):
  A = np.zeros([nAO,nAO,nAO,nAO])
  for a in range(len(X)): # it goes over all the lines of the input array
    mu=int(X[a,0]-1)
    nu=int(X[a,1]-1)
    lam=int(X[a,2]-1)
    sig=int(X[a,3]-1)
    A[mu,nu,lam,sig]=X[a,4]
    A[nu,mu,lam,sig]=X[a,4]
    A[mu,nu,sig,lam]=X[a,4]
    A[nu,mu,sig,lam]=X[a,4]
    A[lam,sig,mu,nu]=X[a,4]
    A[sig,lam,mu,nu]=X[a,4]
    A[lam,sig,nu,mu]=X[a,4]
    A[sig,lam,nu,mu]=X[a,4]
  return A

Eri = make_4Dtensor(eri)

## **Orthogonalisation of the Basis Set : The $\mathbf{S^{-1/2}}$ Matrix**

### Step 4: **Build the Orthogonalization Matrix**

Diagonalize the overlap matrix $$ \mathbf{S} \hspace{1pt} \mathbf{L_S^{\phantom{}}}= \mathbf{L_S^{\phantom{}}} \hspace{1pt} \mathbf{\Lambda_S^{\phantom{}}} $$ $$\implies \mathbf{\Lambda_S^{\phantom{}}} = \mathbf{L_S^\mathsf{T}} \hspace{3pt} \mathbf{S} \hspace{2pt} \mathbf{L_S^{\phantom{}}} $$ where $\mathbf{L_S^{\phantom{}}}$ is the matrix of eigenvectors (columns) and $\mathbf{\Lambda_S}^{\phantom{}}$ is the diagonal matrix of corresponding eigenvalues.
$$$$
Build the symmetric orthogonalization matrix using $$ \mathbf{S^{-1/2}} = \mathbf{L_S^{\phantom{}}} \hspace{3pt} \mathbf{\Lambda_S^{-1/2}} \hspace{3pt} \mathbf{L_S^\mathsf{T}}$$ where the $\hspace{4pt} \mathbf{^\mathsf{T}} $ denotes the matrix transpose.

In [68]:
S_eigval, S_eigvec = np.linalg.eig(S)

lam_S = np.dot(np.transpose(S_eigvec), np.dot(S, S_eigvec)) #we can also use matrix inverse instead of transpose 

def powerofdiagonalmatrix(X,n): # to ^n only to the diagonal elements of the matrix
  for i in range(len(X)):
    X[i,i] = X[i,i]**n
  return X

S_minhalf = np.dot(S_eigvec, np.dot(powerofdiagonalmatrix(lam_S,(-1/2)), np.transpose(S_eigvec))) #we can also use matrix inverse instead of transpose 

print("\n The S\u207b\u00b9\u2e0d\u00b2 matrix for the ", molecule, " test case is \n")
my_print(S_minhalf)
print("\n",end="")


 The S⁻¹⸍² matrix for the  H₂O  test case is 

  1.0236346  -0.1368547   0.0000000  -0.0074873   0.0000000   0.0190279   0.0190279
 -0.1368547   1.1578632  -0.0000000   0.0721601   0.0000000  -0.2223326  -0.2223326
  0.0000000  -0.0000000   1.0733148  -0.0000000   0.0000000  -0.1757583   0.1757583
 -0.0074873   0.0721601  -0.0000000   1.0383050   0.0000000  -0.1184626  -0.1184626
  0.0000000   0.0000000   0.0000000   0.0000000   1.0000000   0.0000000   0.0000000
  0.0190279  -0.2223326  -0.1757583  -0.1184626   0.0000000   1.1297234  -0.0625975
  0.0190279  -0.2223326   0.1757583  -0.1184626   0.0000000  -0.0625975   1.1297234



## **The Initial (Guess) Density Matrix**

### Step 5: **Build the Initial Guess Density**

Form an initial (guess) Fock matrix in the orthonormal AO basis using the core Hamiltonian as a guess $$\mathbf{F^{\prime}}^0 = \mathbf{S^{-1/2}}^\mathsf{T} \hspace{3pt} \mathbf{H^{core}} \hspace{2pt} \mathbf{S^{-1/2}}$$ 
$$$$
Diagonalize the Fock matrix $$\mathbf{F^{\prime}}^0 \hspace{2pt} \mathbf{C^\prime}^0 = \mathbf{C^\prime}^0 \hspace{2pt} \boldsymbol{\epsilon}^0$$ $$\implies \boldsymbol{\epsilon}^0 = \mathbf{C^{\prime\mathsf{T}}}^0 \hspace{3pt} \mathbf{F^{\prime}}^0 \hspace{2pt} \mathbf{C^\prime}^0$$ Note that the $\boldsymbol{\epsilon}^0$ matrix contains the initial MO energies.
$$$$
Transform the eigenvectors into the original (non-orthogonal) AO basis $$\mathbf{C}^0=\mathbf{S^{-1/2}} \hspace{2pt} \mathbf{C^\prime}^0$$
$$$$
Build the density matrix using the occupied MOs $$D_{\mu\nu}^0 = \sum_a^{occ} \hspace{2pt} C^0_{\mu a} \hspace{1pt} C^0_{\nu a}$$  where $\ \ _a\ $ indexes the columns of the coefficient matrices, and the summation includes only the occupied spatial MOs.


In [69]:
F_0prime = np.dot(np.transpose(S_minhalf), np.dot(H_core, S_minhalf))

print("\n The initial Fock matrix in the orthogonal AO basis for the ", molecule, " test case is \n")
my_print(F_0prime)
print("\n",end="")


#F_0prime_eigval, F_0prime_eigvec = np.linalg.eig(F_0prime)    #we are'nt using this, as we want eigenvalues/eigenvectors in a definite(ascending/descending) order
def ascending_eigen(A):
  eigenValues, eigenVectors = np.linalg.eig(A)
  idx = np.argsort(eigenValues) #get the sorting-indexes(ascending) of the eigenvalues
  eigenValues = eigenValues[idx] #order the eigenvalues according to the sorting-indexes(ascending)
  eigenVectors = eigenVectors[:, idx] #order the eigenvectors according to the sorting-indexes(ascending)
  return (eigenValues, eigenVectors)

F_0prime_eigval, F_0prime_eigvec = ascending_eigen(F_0prime) # get eigenvalues/eigenvectors in a ascending order

epsilon_0 = np.dot(np.transpose(F_0prime_eigvec), np.dot(F_0prime, F_0prime_eigvec)) #we can also use matrix inverse instead of transpose 

C_0 = np.dot(S_minhalf, F_0prime_eigvec)

print("\n The initial MO coefficient matrix for the ", molecule, " test case is \n")
my_print(C_0)
print("\n",end="")


D_0 = np.zeros([nAO,nAO])

for mu in range(nAO):
  for nu in range(nAO):
    for a in range(int(N/2)):
      D_0[mu,nu] += C_0[mu,a]*C_0[nu,a]

print("\n The initial density matrix for the ", molecule, " test case is \n")
my_print(D_0)
print("\n",end="")


 The initial Fock matrix in the orthogonal AO basis for the  H₂O  test case is 

-32.2545866  -2.7914909  -0.0000000   0.0086110   0.0000000  -0.1812967  -0.1812967
 -2.7914909  -8.2368891   0.0000000  -0.2282926  -0.0000000  -0.3857987  -0.3857987
 -0.0000000   0.0000000  -7.5428890   0.0000000   0.0000000  -0.1132121   0.1132121
  0.0086110  -0.2282926   0.0000000  -7.4570295  -0.0000000  -0.1102196  -0.1102196
  0.0000000  -0.0000000   0.0000000  -0.0000000  -7.3471449  -0.0000000   0.0000000
 -0.1812967  -0.3857987  -0.1132121  -0.1102196  -0.0000000  -4.0329547  -0.0446466
 -0.1812967  -0.3857987   0.1132121  -0.1102196   0.0000000  -0.0446466  -4.0329547


 The initial MO coefficient matrix for the  H₂O  test case is 

  1.0015436  -0.2336245   0.0000000  -0.0856842  -0.0000000   0.0482226   0.0000000
 -0.0071893   1.0579388  -0.0000000   0.3601105   0.0000000  -0.4631213  -0.0000000
  0.0000000  -0.0000000  -1.0610702  -0.0000000   0.0000000   0.0000000  -0.2965071
 -0.0002671 

### Step 6: **Compute the Inital SCF Energy**

The SCF electronic energy may be computed using the density matrix as $$E^0_{elec} = \sum_{\mu\nu}^{AO} D^0_{\mu\nu} \hspace{1pt} \Big( H^{core}_{\mu\nu} + H^{core}_{\mu\nu} \Big)$$ 
$$$$
The total energy is the sum of the electronic energy and the nuclear repulsion energy $$E^0_{total} = E^0_{elec} + E_{nuc}$$ where $\ \ ^0 $ denotes the initial SCF energy.


In [70]:
E_elec = []
E_total = []

i=0  # iteration counter
E_elec.append(0)
E_total.append(0)

for mu in range(nAO):
  for nu in range(nAO):
    E_elec[i] += D_0[mu,nu] * ( H_core[mu,nu] + H_core[mu,nu] )

E_total[i] = E_elec[i] + E_nuc

print("\n The initial Hartree-Fock electronic energy for the ", molecule, " test case is {:18.12f} Hartrees. \n".format(E_elec[0]))


 The initial Hartree-Fock electronic energy for the  H₂O  test case is  -125.842077437699 Hartrees. 



## **The Self-Consistent Field Iteration**

### Step 7: **Compute the New Fock Matrix**

Start the SCF iterative procedure by building a new Fock matrix using the previous iteration's density as $$F^i_{\mu\nu} = H_{\mu\nu}^{core} + \sum_{\lambda\sigma}^{AO} \hspace{2pt} D_{\lambda\sigma}^{i-1} \hspace{2pt} \Big[ 2(\mu\nu|\lambda\sigma) - (\mu\lambda|\nu\sigma) \Big]$$ where the double-summation runs over all the AOs and $\ \ ^{i-1} $ denotes the density for the last iteration.

In [71]:
F = np.zeros([nAO,nAO])

def get_new_F(H_core,D,Eri):
  F = np.zeros([nAO,nAO])
  for mu in range(nAO):
    for nu in range(nAO):
      F[mu,nu] = H_core[mu,nu]
      for lam in range(nAO):
        for sig in range(nAO):
          F[mu,nu] += D[lam,sig]*( 2*Eri[mu,nu,lam,sig] - Eri[mu,lam,nu,sig] )
  return F

F = get_new_F(H_core,D_0,Eri)

print("\n The first-iteration Fock matrix in the AO basis for the ", molecule, " test case is \n")
my_print(F)
print("\n",end="")


 The first-iteration Fock matrix in the AO basis for the  H₂O  test case is 

-18.8132695  -4.8726875  -0.0000000  -0.0115290   0.0000000  -0.8067323  -0.8067323
 -4.8726875  -1.7909029  -0.0000000  -0.1808692   0.0000000  -0.5790557  -0.5790557
 -0.0000000  -0.0000000   0.1939644   0.0000000   0.0000000  -0.1708886   0.1708886
 -0.0115290  -0.1808692   0.0000000   0.2391247   0.0000000  -0.1828683  -0.1828683
  0.0000000   0.0000000   0.0000000   0.0000000   0.3091071   0.0000000   0.0000000
 -0.8067323  -0.5790557  -0.1708886  -0.1828683   0.0000000  -0.1450338  -0.1846675
 -0.8067323  -0.5790557   0.1708886  -0.1828683   0.0000000  -0.1846675  -0.1450338



### Step 8: **Build the New Density Matrix**

Form the new density matrix following the same procedure as in Step 5 above.

Orthogonalize $$\mathbf{F^\prime}^i = \mathbf{S^{-1/2}}^\mathsf{T} \hspace{2pt} \mathbf{F}^i \hspace{2pt} \mathbf{S^{-1/2}}$$

Diagonalize $$\mathbf{F^\prime}^i \hspace{2pt} \mathbf{C^\prime}^i=\mathbf{C^\prime}^i \hspace{2pt} \boldsymbol{\epsilon}^i$$

Back-transform $$\mathbf{C}^i=\mathbf{S^{-1/2}} \hspace{3pt} \mathbf{C^\prime}^i$$

Compute the density $$D_{\mu\nu}^i = \sum_a^{occ} \hspace{2pt} C^i_{\mu a} \hspace{1pt} C^i_{\nu a}$$ where $\ \ ^i $ denotes the current iteration density.

In [72]:
def get_new_D(F):
  F_prime = np.dot(np.transpose(S_minhalf), np.dot(F, S_minhalf)) #we can also use matrix inverse instead of transpose 
  
  F_prime_eigval, F_prime_eigvec = ascending_eigen(F_prime) # get eigenvalues/eigenvectors in a ascending order

  epsilon = np.dot(np.transpose(F_prime_eigvec), np.dot(F_prime, F_prime_eigvec)) #we can also use matrix inverse instead of transpose 

  C = np.dot(S_minhalf, F_prime_eigvec)


  D = np.zeros([nAO,nAO])

  for mu in range(nAO):
    for nu in range(nAO):
      for a in range(int(N/2)):
        D[mu,nu] += C[mu,a]*C[nu,a]
  
  return epsilon, C, D

epsilon, C, D = get_new_D(F)

### Step 9: **Compute the New SCF Energy**

Compute the new SCF energy as in step 6 above $$E_{elec}^i = \sum_{\mu\nu}^{occ} \hspace{2pt} D^i_{\mu\nu} \hspace{1pt} \Big( H^{core}_{\mu\nu} + F^i_{\mu\nu} \Big)$$

$$E_{total}^i = E_{elec}^i + E_{nuc}$$
 where $\ \ ^i $ denotes the SCF energy for the ith iteration.


In [73]:
def get_new_E_elec(D,H_core,F,i):
  E_elec.append(0)
  for mu in range(nAO):
    for nu in range(nAO):
      E_elec[i] += D[mu,nu]*(H_core[mu,nu]+F[mu,nu])
  return E_elec

def get_new_E_total(E_elec,E_nuc,i):
  E_total.append(0)
  E_total[i] = E_elec[i] + E_nuc
  return E_total

i=1
E_elec = get_new_E_elec(D,H_core,F,i)
E_total = get_new_E_total(E_elec,E_nuc,i)

### Step 10: **Test for Convergence**

Test both the energy and the density for convergence $$\Delta E^i = E_{elec}^i - E_{elec}^{i-1} < \delta_1$$

$${rms_D}^i = \left[ \sum_{\mu\nu} \Big( D_{\mu\nu}^i - D_{\mu\nu}^{i-1} \Big)^2 \right]^{1/2} < \delta_2$$

If the difference in consecutive SCF energy and the root-mean-squared difference in consecutive densities do not fall below the prescribed thresholds, return to Step 7 and continue from there.

In [74]:
Delta_E = []
Delta_E.append("") # 0th element is empty
rms_D = []
rms_D.append("") # 0th element is empty

def get_new_Delta_E(E_elec,i):
  Delta_E.append(0)
  Delta_E[i] = E_elec[i] - E_elec[i-1]
  return Delta_E

def get_new_rms_D(D_old,D,i):
  rms_D.append(0)
  for mu in range(nAO):
    for nu in range(nAO): 
      rms_D[i] += ( D[mu,nu] - D_old[mu,nu] )**2
  rms_D[i] = rms_D[i]**(1/2)
  return rms_D

i=1
D_old = D_0
Delta_E = get_new_Delta_E(E_elec,i)
rms_D = get_new_rms_D(D_old,D,i)



while rms_D[i]>1e-11:
  F = get_new_F(H_core,D,Eri)
  D_old = D
  epsilon, C, D = get_new_D(F)
  i += 1
  E_elec = get_new_E_elec(D,H_core,F,i)
  E_total = get_new_E_total(E_elec,E_nuc,i)
  Delta_E = get_new_Delta_E(E_elec,i)
  rms_D = get_new_rms_D(D_old,D,i)


print("\n The iteration-by-iteration electronic and total energies for the ", molecule, " test case are as follows \n")
df = pd.DataFrame(list(zip(E_elec,E_total,Delta_E,rms_D)), columns=['E(elec)','E(tot)','Delta(E)','RMS(D)'])
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:18.12f}'.format
pd.set_option('display.colheader_justify', 'center')
print(df)
print("\n",end="")


 The iteration-by-iteration electronic and total energies for the  H₂O  test case are as follows 

         E(elec)            E(tot)            Delta(E)            RMS(D)      
0   -125.842077437699  -117.839710375888                                      
1    -78.286583284740   -70.284216222929    47.555494152959     1.826673084479
2    -84.048316253435   -76.045949191625    -5.761732968696     0.479570364860
3    -82.716965960854   -74.714598899044     1.331350292581     0.086831688906
4    -82.987140757001   -74.984773695191    -0.270174796147     0.031026136359
5    -82.938133187513   -74.935766125703     0.049007569488     0.010799283179
6    -82.946271078699   -74.943904016889    -0.008137891186     0.005254826831
7    -82.944486784914   -74.942119723104     0.001784293785     0.002438579642
8    -82.944617252243   -74.942250190433    -0.000130467329     0.001177279531
9    -82.944503500703   -74.942136438892     0.000113751541     0.000564543180
10   -82.944478930625   -74.942

## **Additional Concepts**

### **The MO-Basis Fock Matrix**

At convergence, the canonical Hartree-Fock MOs are, by definition, eigenfunctions of the Fock operator, viz. $$F \chi_i = \epsilon_i \chi_i$$

If we multiply on the left by an arbitrary MO and integrate, we obtain $$F_{ij} \equiv \epsilon_i \delta_i = \langle \chi_j | F | \chi_i \rangle$$

In other words, the Fock matrix should be diagonal in the MO basis, with the orbital energies as its diagonal elements. We can demonstrate this explicitly using the AO-basis Fock matrix by first re-writing the above expression using the LCAO-MO coefficients $$F_{ij} = \sum_{\mu\nu} C_{\mu j} C_{\nu i} \langle \phi_\mu | F | \phi_\nu \rangle = \sum_{\mu\nu} C_{\mu j} C_{\nu i} F_{\mu\nu}$$

Use the above equation to transform the Fock matrix from the AO basis to the MO basis and demonstrate that it is indeed diagonal (to within the convergence limits of the SCF iterative procedure).

In [75]:
F_MO = np.zeros([nAO,nAO])

for i in range(nAO):
  for j in range(nAO):

    for mu in range(nAO):
      for nu in range(nAO):

        F_MO[i,j] += C[mu,j]*C[nu,i]*F[mu,nu]


print("\n Fock matrix in MO basis is \n")
my_print(F_MO) #actually same as epsilon
print("\n",end="")


 Fock matrix in MO basis is 

-20.2628916  -0.0000000  -0.0000000   0.0000000  -0.0000000   0.0000000   0.0000000
 -0.0000000  -1.2096974   0.0000000   0.0000000   0.0000000   0.0000000  -0.0000000
 -0.0000000   0.0000000  -0.5479646   0.0000000  -0.0000000   0.0000000  -0.0000000
  0.0000000   0.0000000   0.0000000  -0.4365272   0.0000000  -0.0000000   0.0000000
 -0.0000000   0.0000000  -0.0000000   0.0000000  -0.3875867  -0.0000000   0.0000000
  0.0000000   0.0000000   0.0000000  -0.0000000  -0.0000000   0.4776187  -0.0000000
  0.0000000   0.0000000   0.0000000  -0.0000000   0.0000000  -0.0000000   0.5881393



### **One-Electron Properties**

The calculation of one-electron properties requires density matrix and the relevant property integrals. The electronic contribution to the electric-dipole moment may be computed using, $$\vec{\mu}_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \vec{\mu}_{elec} | \phi_{\nu} \rangle $$ where the vector notation implies three sets of dipole-moment integrals -- one for each Cartesian component of the dipole operator. $$\mu^x_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^x_{elec} | \phi_{\nu} \rangle \quad \mu^y_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^y_{elec} | \phi_{\nu} \rangle \quad \mu^z_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^z_{elec} | \phi_{\nu} \rangle $$ The factor 2 appearing above arises because the definition of the density used in this project differs from that used in Szabo & Ostlund.
$$$$
In order to compute the total dipole moment, we must include the nuclear contribution, which requires the atomic numbers $Z$ ( ≡ charge $q$) and Cartesian coordinates $\vec{r}$ of the nuclei $$\vec{\mu}_{nuc} = \sum_i \hspace{1pt} q_i \hspace{2pt} (\vec{r}_i - \vec{r}_{cm})$$ where $\vec{r}_{cm}$ is the Cartesian coordiate of centre-of-mass.
$$$$
Hence the total dipole moment $$\vec{\mu} = \vec{\mu}_{elec} + \vec{\mu}_{nuc}$$

In [76]:
masses = np.array([0.,1.00782503207,4.00260325415,7.016004548,9.012182201,11.009305406, 12,14.00307400478,15.99491461956,18.998403224,19.99244017542,
22.98976928087,23.985041699,26.981538627,27.97692653246,30.973761629,31.972070999,34.968852682,39.96238312251,38.963706679,39.962590983,44.955911909,
47.947946281,50.943959507,51.940507472,54.938045141,55.934937475,58.933195048,57.935342907,62.929597474,63.929142222,68.925573587,73.921177767,
74.921596478,79.916521271,78.918337087,85.910610729,84.911789737,87.905612124,88.905848295,89.904704416,92.906378058,97.905408169,98.906254747,
101.904349312,102.905504292,105.903485715,106.90509682,113.90335854,114.903878484,119.902194676,120.903815686,129.906224399,126.904472681,131.904153457,
132.905451932,137.905247237,138.906353267,139.905438706,140.907652769,141.907723297,144.912749023,151.919732425,152.921230339,157.924103912,158.925346757,
163.929174751,164.93032207,165.930293061,168.93421325,173.938862089,174.940771819,179.946549953,180.947995763,183.950931188,186.955753109,191.96148069,
192.96292643,194.964791134,196.966568662,201.970643011,204.974427541,207.976652071,208.980398734,208.982430435,210.987496271,222.017577738,222.01755173,
228.031070292,227.027752127,232.038055325,231.03588399,238.050788247,237.048173444,242.058742611,243.06138108,247.07035354,247.07030708,251.079586788,
252.082978512,257.095104724,258.098431319,255.093241131,260.105504,263.112547,255.107398,259.114500,262.122892,263.128558,265.136151,281.162061,272.153615,
283.171792,283.176451,285.183698,287.191186,292.199786,291.206564,293.214670]) #data taken from https://github.com/psi4/psi3/blob/master/include/masses.h 😏

def add_mass_column(geom): # to dd a column to the geom, for the mass of the atoms
  geom = np.c_[geom, np.zeros(len(geom))]  # add an all-zero column to our geom array
  for i in range(len(geom)): 
    geom[i,4] = masses[int(geom[i,0])]
  return geom

def center_of_mass(geom):
  cm = np.zeros([3])
  xcm_num=0
  ycm_num=0
  zcm_num=0
  denom=0
  for i in range(len(geom)):
    xcm_num += geom[i,4]*geom[i,1]
    ycm_num += geom[i,4]*geom[i,2]
    zcm_num += geom[i,4]*geom[i,3]
    denom += geom[i,4]
  cm[0] = xcm_num/denom
  cm[1] = ycm_num/denom
  cm[2] = zcm_num/denom
  return cm


Muxmunu = np.genfromtxt("mux.dat.txt")
Muymunu = np.genfromtxt("muy.dat.txt")
Muzmunu = np.genfromtxt("muz.dat.txt")

Mux = make_2Dmatrix(Muxmunu)
Muy = make_2Dmatrix(Muymunu)
Muz = make_2Dmatrix(Muzmunu)

def electronic_dipolemoment(D,Mux,Muy,Muz):
  Dipole_elec = np.zeros([3])
  
  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[0] += D[mu,nu]*Mux[mu,nu]
  Dipole_elec[0] *= 2

  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[1] += D[mu,nu]*Muy[mu,nu]
  Dipole_elec[1] *= 2

  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[2] += D[mu,nu]*Muz[mu,nu]
  Dipole_elec[2] *= 2

  return Dipole_elec

Dipole_elec = electronic_dipolemoment(D,Mux,Muy,Muz)


geom3 = np.genfromtxt("geom.dat.txt", skip_header=1)
geom3=add_mass_column(geom3) # now the geom array has a 4th column, for the mass of the atoms

cm = center_of_mass(geom3)

def nuclear_dipolemoment(geom):
  Dipole_nuc = np.zeros([3])
  for i in range(3): #for each coordinate
    for A in range(len(geom)): #for each atom
      Dipole_nuc[i] += geom[A,0]*(geom[A,i+1]-cm[i])
  return Dipole_nuc

Dipole_nuc = nuclear_dipolemoment(geom3)


Dipole = np.add(Dipole_elec, Dipole_nuc)


print("\n Dipole moment \n")
print("\u03bc\u02e3 =  {:16.12f} \n\u03bc\u02b8 =  {:16.12f} \n\u03bc\u1dbb =  {:16.12f} \n".format(Dipole[0],Dipole[1],Dipole[2]))


 Dipole moment 

μˣ =    0.000000000000 
μʸ =    0.603521296526 
μᶻ =   -0.000000000000 



### **Population Analysis/Atomic Charges**

A Mulliken population analysis requires the overlap integrals and the electron density, in addition to information about the number of basis functions centered on each atom. The charge on atom A may be computed as $$q_A = Z_A - 2 \sum_{\mu\in A} (\mathbf{DS})_{\mu\mu}$$ where the summation is limited to only those basis functions centered on atom A.

In [77]:
def sum_arr_upto(X,n): # X is the 1D array, n is upto how many elements we want the sum
  sum = 0
  for i in range(n):
    sum += X[i]
  return sum


q = np.zeros([len(geom3)])

print("\n Atomic charges \n")
for A in range(len(geom3)):
  thesum = 0
  for mu in range(sum_arr_upto(n_basis,A),sum_arr_upto(n_basis,A+1)):
    thesum += np.dot(D,S)[mu,mu]
  q[A] = geom3[A,0] - 2*thesum
  print("Charge on atom {:1.0f} =   {:16.12f}".format(A,q[A]))
print("\n",end="")


 Atomic charges 

Charge on atom 0 =    -0.253146052405
Charge on atom 1 =     0.126573026202
Charge on atom 2 =     0.126573026202



# Project 4: **The second-order Moller-Plesset perturbation (MP2) energy**

Unlike the Hartree-Fock energy, correlation energies like the MP2 energy are usually expressed in terms of MO-basis quantities (integrals, MO energies).

### Step 1: **Read the Two-Electron Integrals**


Same as Step 3 of Project 3

### Step 2: **Obtain the MO Coefficients and MO Energies**


The MO Coefficients values $C_{\mu p}$ and MO energies $\epsilon_p$ computed in the Hartree-Fock program of Project 3.

### Step 3: **Transform the Two-Electron Integrals to the MO Basis**


This is the most expensive part of the calculation.
$$$$

**Noddy Algorithm**

The most straightforward expression of the AO/MO integral transformation is $$(pq|rs) = \sum_\mu \sum_\nu \sum_\lambda \sum_\sigma C_{\mu p} \hspace{1pt} C_{\nu q} \hspace{1pt} (\mu\nu|\lambda\sigma) \hspace{1pt} C_{\lambda r} \hspace{1pt} C_{\sigma s}$$ This approach is easy to implement, but is expensive due to its $\mathcal{O(n^8)}$ computational order.

$$$$

**The Smarter Algorithm**

Notice that none of the $C$ coefficients in the above expression have any indices in common. Thus, the summation could be rearranged such that $$(pq|rs) = \sum_\mu C_{\mu p} \Bigg[ \sum_\nu C_{\nu q} \Bigg[ \sum_\lambda C_{\lambda r} \Bigg[ \sum_\sigma C_{\sigma s} \hspace{1pt} (\mu\nu|\lambda\sigma) \Bigg] \Bigg] \Bigg]$$ This means that each summation within brackets could be carried out separately, if we store the results at each step. This reduces the $\mathcal{O(n^8)}$ algorithm above to four $\mathcal{O(n^5)}$ steps.

In [78]:
Eri_MO = np.zeros([nAO,nAO,nAO,nAO])

for p in range(nAO):
  for q in range(nAO):
    for r in range(nAO):
      for s in range(nAO):

        for mu in range(nAO):
          for nu in range(nAO):
            for lam in range(nAO):
              for sig in range(nAO):

                Eri_MO[p,q,r,s] += C[mu,p]*C[nu,q]*C[lam,r]*C[sig,s]*Eri[mu,nu,lam,sig]


### Step 4: **Compute the MP2 Energy**

$$E_{MP2} = \sum_{ij} \sum_{ab} \frac{(ia|jb) \hspace{1pt} \Big[ 2(ia|jb) - (ib|ja) \Big]}{\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b}$$ 
where $i$ and $j$ denote doubly-occupied orbitals and $a$ and $b$ unoccupied orbitals, and the denominator involves the MO energies.

In [79]:
Emp2 = 0

for i in range(int(N/2)):
  for j in range(int(N/2)):

    for a in range(int(N/2),nAO):
      for b in range(int(N/2),nAO):

        Emp2 += Eri_MO[i,a,j,b] * (2*Eri_MO[i,a,j,b]-Eri_MO[i,b,j,a]) / (epsilon[i,i]+epsilon[j,j]-epsilon[a,a]-epsilon[b,b])


Escf = E_total[-1]
Etot = Escf + Emp2

print("\n Escf =  {:18.12f}".format(Escf))
print(" Emp2 =  {:18.12f}".format(Emp2))
print("\u23af"*30)
print(" Etot =  {:18.12f} \n".format(Etot))


 Escf =    -74.942079928192
 Emp2 =     -0.049149636120
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
 Etot =    -74.991229564312 

